In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# set a random seed for reproducibility
np.random.seed(42)

In [ ]:
# function to evaluate loss
def evaluate_loss(X_test_scaled, y_test_scaled, model):
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test_scaled, y_pred)
    print(f'Mean Squared Error on Test Set: {mse}')

In [ ]:
# Load the train dataset
train_dataset = pd.read_excel('/home/framework/coding_python/wqi_ann/src/data_input/train_dataset.xlsx')  # replace with the path to your excel file


In [ ]:
# separate features (11 variables) and target (WQI)
x = train_dataset[['ph', 'Total  Hardness', 'Total Alkalinity', 'Cl', 'NO3', 'SO4', 'F', 'TDS', 'Fe', 'As (ppb)', 'Pb (ppb)']]
y = train_dataset[['WQI']]


In [ ]:
# normalize the features and target using MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)


In [ ]:
# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)


In [ ]:
# build an XGBoost model with more customizable parameters
model = XGBRegressor(n_estimators=300,      # Number of boosting rounds
                     learning_rate=0.05,    # Step size shrinkage
                     max_depth=6,           # Depth of each tree
                     subsample=0.8,         # Fraction of samples used for training each tree
                     colsample_bytree=0.8,  # Fraction of features used per tree
                     reg_alpha=0.1,         # L1 regularization term
                     reg_lambda=0.5,        # L2 regularization term
                     random_state=42)


In [ ]:
# train the model
model.fit(X_train, y_train.ravel())


In [ ]:
# load the test dataset (11 variables without WQI)
test_dataset = pd.read_excel('/home/framework/coding_python/wqi_ann/src/data_input/test_dataset.xlsx')


In [ ]:
# extract the 11 variables
X_test_data = test_dataset[['pH', 'Total Hardness', 'Total Alkalinity', 'Cl', 'NO3', 'SO4', 'F', 'TDS', 'Fe', 'As (ppb)', 'Pb (ppb)']]


In [ ]:
# normalize the test data using the same scaler fitted to the training data
X_test_data_scaled = scaler_X.transform(X_test_data)


In [ ]:
# Predict WQI values using the trained model
predicted_wqi_scaled = model.predict(X_test_data_scaled)


In [ ]:
# print scaled prediction using the trained model
print("Scaled prediction:", predicted_wqi_scaled)


In [ ]:
# Inverse transform to get the original WQI values
predicted_wqi = scaler_y.inverse_transform(predicted_wqi_scaled.reshape(-1, 1))


In [ ]:
# print unscaled prediction for debugging
print("Unscaled predictions:", predicted_wqi)


In [ ]:
# Evaluate the loss of the model on the test set
evaluate_loss(X_test, y_test, model)


In [ ]:
# Add the predicted WQI values to the test dataset
test_dataset['WQI'] = predicted_wqi

In [ ]:
# Save the updated test dataset to a new Excel file
output_excel_path = '/home/framework/coding_python/wqi_ann/src/data_output/validation_dataset_output.xlsx'
test_dataset.to_excel(output_excel_path, index=False)


In [ ]:
print("WQI values have been added and saved to the output Excel file.")